# MVP Pipeline de Dados
## Pesquisa sobre aparelhos celulares

Edmilson Prata da Silva

PUC-RJ - MBA em Ciência de Dados e Analytics

Disciplina de Engenharia de Dados

## Script ETL para carga na camada GOLD

## Imports

Imports das bibliotecas necessárias para o funcionamento do script.

In [0]:
import re
import uuid
import warnings
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

## Carga de Dados

Os dados serão carregados a partir da camada silver gerando as métricas a serem persistidas na camada gold.

In [0]:
spk_company_metrics = spark.sql(""" 
    SELECT c.company_id
         , c.company_name
         , INT(AVG(p.launched_price)) AS mean_price
         , INT(MAX(p.launched_price)) AS max_price
         , INT(MIN(p.launched_price)) AS min_price
         , INT(AVG(m.battery_capacity)) AS mean_battery
         , INT(MIN(m.battery_capacity)) AS min_battery
         , INT(MAX(m.battery_capacity)) AS max_battery
         , AVG(m.screen_size) AS mean_screen_size
         , MIN(m.screen_size) AS min_screen_size
         , MAX(m.screen_size) AS max_screen_size
         , AVG(m.ram) AS mean_ram
         , MIN(m.ram) AS min_ram
         , MAX(m.ram) AS max_ram
    FROM silver.smartphones s
    JOIN silver.company c on c.company_id = s.company_id
    JOIN silver.model m ON m.model_id = s.model_id
    JOIN silver.price p ON p.model_id = m.model_id
     and p.country = 'usa'
   GROUP BY c.company_id, c.company_name;     
""")

spk_company_metrics.printSchema()

root
 |-- company_id: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- mean_price: integer (nullable = true)
 |-- max_price: integer (nullable = true)
 |-- min_price: integer (nullable = true)
 |-- mean_battery: integer (nullable = true)
 |-- min_battery: integer (nullable = true)
 |-- max_battery: integer (nullable = true)
 |-- mean_screen_size: decimal(8,6) (nullable = true)
 |-- min_screen_size: decimal(4,2) (nullable = true)
 |-- max_screen_size: decimal(4,2) (nullable = true)
 |-- mean_ram: double (nullable = true)
 |-- min_ram: integer (nullable = true)
 |-- max_ram: integer (nullable = true)



In [0]:
spk_model_metrics = spark.sql("""
    SELECT * FROM (
        SELECT m.model_id
             , m.model_name
             , c.company_id
             , c.company_name
             , ROW_NUMBER() OVER (ORDER BY m.battery_capacity DESC) ranking
             , 'Battery Capacity' characteristic_label
             , 'mAh' characteristic_unit
             , m.battery_capacity characteristic_value
          FROM silver.smartphones s
          JOIN silver.company c 
            ON c.company_id = s.company_id
          JOIN silver.model m 
            ON m.model_id = s.model_id
         ORDER BY m.battery_capacity DESC
         LIMIT 10
    )

    UNION ALL

    SELECT * FROM (
        SELECT m.model_id
             , m.model_name
             , c.company_id
             , c.company_name
             , ROW_NUMBER() OVER (ORDER BY m.ram DESC) ranking
             , 'RAM' characteristic_label
             , 'GB' characteristic_unit
             , m.ram characteristic_value
          FROM silver.smartphones s
          JOIN silver.company c 
            ON c.company_id = s.company_id
          JOIN silver.model m 
            ON m.model_id = s.model_id
         ORDER BY m.ram DESC
         LIMIT 10
    )

    UNION ALL

    SELECT * FROM (
        SELECT m.model_id
             , m.model_name
             , c.company_id
             , c.company_name
             , ROW_NUMBER() OVER (ORDER BY m.back_camera DESC) ranking
             , 'Back Camera' characteristic_label
             , 'MP' characteristic_unit
             , m.back_camera characteristic_value
         FROM silver.smartphones s
         JOIN silver.company c 
           ON c.company_id = s.company_id
         JOIN silver.model m 
           ON m.model_id = s.model_id
        ORDER BY m.ram DESC
        LIMIT 10
    )
""")

spk_model_metrics.printSchema()

root
 |-- model_id: string (nullable = true)
 |-- model_name: string (nullable = true)
 |-- company_id: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- ranking: integer (nullable = false)
 |-- characteristic_label: string (nullable = false)
 |-- characteristic_unit: string (nullable = false)
 |-- characteristic_value: decimal(12,2) (nullable = true)



In [0]:
spk_segmentation_metrics = spark.sql("""
    SELECT uuid() segmentation_code
        , 'Segmentação por Preços' segmentation_name
        , '< $200' range1_name
        , '$200-$500' range2_name
        , '> $500' range3_name
        , (SELECT COUNT(1) FROM silver.price p WHERE p.launched_price < 200) range1_qtd
        , (SELECT COUNT(1) FROM silver.price p WHERE p.launched_price BETWEEN 200 AND 500) range2_qtd
        , (SELECT COUNT(1) FROM silver.price p WHERE p.launched_price > 500) range3_qtd

    UNION ALL

    SELECT uuid() segmentation_code
        , 'Segmentação por Tela' segmentation_name
        , 'Pequena' range1_name
        , 'Média' range2_name
        , 'Grande' range3_name
        , (SELECT COUNT(1) FROM silver.model m WHERE m.screen_size < 6) range1_qtd
        , (SELECT COUNT(1) FROM silver.model m WHERE m.screen_size BETWEEN 6 AND 6.5) range2_qtd
        , (SELECT COUNT(1) FROM silver.model m WHERE m.screen_size > 6.5) range3_qtd

    UNION ALL

    SELECT uuid() segmentation_code
        , 'Segmentação por RAM (capacidade)' segmentation_name
        , 'Baixa' range1_name
        , 'Média' range2_name
        , 'Alta' range3_name
        , (SELECT COUNT(1) FROM silver.model m WHERE m.ram <= 4) range1_qtd
        , (SELECT COUNT(1) FROM silver.model m WHERE m.ram BETWEEN 4.1 AND 8) range2_qtd
        , (SELECT COUNT(1) FROM silver.model m WHERE m.ram > 8) range3_qtd
""")

spk_segmentation_metrics.printSchema()

root
 |-- segmentation_code: string (nullable = false)
 |-- segmentation_name: string (nullable = false)
 |-- range1_name: string (nullable = false)
 |-- range2_name: string (nullable = false)
 |-- range3_name: string (nullable = false)
 |-- range1_qtd: long (nullable = true)
 |-- range2_qtd: long (nullable = true)
 |-- range3_qtd: long (nullable = true)



### Persistência dos dados

Aqui será feita a persistência dos dados nas tabelas da camada gold.

In [0]:
# Grava os dados na tabela Delta:
def save_data(table_name, df_spark):
    df_spark \
    .write \
    .format("delta") \
    .mode("append") \
    .saveAsTable(table_name)
    print(f"Dados salvos com sucesso na tabela {table_name}.")

In [0]:
spk_company_metrics = spk_company_metrics.withColumn("mean_screen_size", col("mean_screen_size").cast("decimal(10,2)"))
spk_company_metrics = spk_company_metrics.withColumn("max_screen_size", col("max_screen_size").cast("decimal(10,2)"))
spk_company_metrics = spk_company_metrics.withColumn("min_screen_size", col("min_screen_size").cast("decimal(10,2)"))
spk_company_metrics = spk_company_metrics.withColumn("mean_ram", col("mean_ram").cast("decimal(10,2)"))
spk_company_metrics = spk_company_metrics.withColumn("max_ram", col("max_ram").cast("decimal(10,2)"))
spk_company_metrics = spk_company_metrics.withColumn("min_ram", col("min_ram").cast("decimal(10,2)"))
save_data('gold.company_metrics', spk_company_metrics)

Dados salvos com sucesso na tabela gold.company_metrics.


In [0]:
spk_model_metrics = spk_model_metrics.withColumn("characteristic_value", col("characteristic_value").cast("decimal(10,2)"))
save_data('gold.model_metrics', spk_model_metrics)

Dados salvos com sucesso na tabela gold.model_metrics.


In [0]:
spk_segmentation_metrics = spk_segmentation_metrics.withColumn("range1_qtd", col("range1_qtd").cast("integer"))
spk_segmentation_metrics = spk_segmentation_metrics.withColumn("range2_qtd", col("range2_qtd").cast("integer"))
spk_segmentation_metrics = spk_segmentation_metrics.withColumn("range3_qtd", col("range3_qtd").cast("integer"))
save_data('gold.segmentation_metrics', spk_segmentation_metrics)

Dados salvos com sucesso na tabela gold.segmentation_metrics.


### Teste de carga da Tabela

Teste de carga da tabela, para garantir o sucesso da operação.

In [0]:
%sql select * from gold.company_metrics limit 10

company_id,company_name,mean_price,max_price,min_price,mean_battery,max_battery,min_battery,mean_screen_size,max_screen_size,min_screen_size,mean_ram,max_ram,min_ram
55db049c-f1c9-4760-b4f3-365a1310dc83,iQOO,399,449,349,8040,8040,8040,11.87,12.30,11.00,7.33,8.00,6.00
fca2bc57-3f4e-4fce-860b-68c09e167af0,Xiaomi,559,999,199,5172,6100,4610,6.62,6.73,6.36,10.52,16.00,4.00
a2d25021-3c44-4554-886f-31d39c9ce5cf,Vivo,400,999,1,4678,8040,2000,6.79,12.90,5.00,7.50,16.00,1.00
0e37c57d-fa83-4e60-bf8e-f58663a7a9d5,Realme,273,500,130,5691,8360,5000,7.43,11.50,6.50,8.13,16.00,3.00
a8b99050-4edd-4f52-be47-f1412f22d271,Huawei,424,999,1,4900,10100,4000,7.17,13.20,6.50,9.71,16.00,8.00
67a5680f-b05e-43bc-bb18-c4e37156c89a,Oppo,441,999,1,5004,9510,4000,6.89,12.10,6.40,9.43,16.00,4.00
90df224a-3a24-4c6d-aba0-055521efeb19,Tecno,283,999,1,5315,8000,4300,6.96,11.00,6.52,7.21,12.00,3.00
c482a035-36fe-4b4a-8a6f-a3b10aa45304,OnePlus,587,899,1,4840,11000,3300,6.89,12.40,5.50,9.43,16.00,4.00
05f2afcc-51bd-412e-87dd-b32ac7e2ff4e,Samsung,381,1719,1,5119,11200,2600,7.33,14.60,5.20,7.82,15.00,2.00
b584fcf4-6fea-46d7-8a5b-2cc994df30f3,Apple,424,999,1,4442,10307,2227,7.14,13.00,5.40,5.33,8.00,3.00


In [0]:
%sql select * from gold.model_metrics limit 10

model_id,model_name,company_id,company_name,ranking,characteristic_label,characteristic_unit,characteristic_value
20853b9e-1b6b-49c5-89a7-64f770425f21,Galaxy Tab S9 Ultra 256GB,05f2afcc-51bd-412e-87dd-b32ac7e2ff4e,Samsung,1,Battery Capacity,mAh,11200.00
c1e1d82d-a630-49c4-a167-4a887a13e55d,Galaxy Tab S8 Ultra 256GB,05f2afcc-51bd-412e-87dd-b32ac7e2ff4e,Samsung,2,Battery Capacity,mAh,11200.00
77b0574a-6b90-4bdc-89ae-b9210e7ea955,OnePlus Pad Pro,c482a035-36fe-4b4a-8a6f-a3b10aa45304,OnePlus,3,Battery Capacity,mAh,11000.00
5ada5cad-04f9-4f00-8f9d-87ef4d95a910,MagicPad 2,6d36605e-cb30-4904-ac48-3f077de29121,Honor,4,Battery Capacity,mAh,10500.00
40f59c1a-330c-447c-814d-aa8dfea5e6a6,Pad GT Pro,6d36605e-cb30-4904-ac48-3f077de29121,Honor,5,Battery Capacity,mAh,10500.00
a60a06f2-72d6-49d2-90b0-84e2deaa8a14,MagicPad 3,6d36605e-cb30-4904-ac48-3f077de29121,Honor,6,Battery Capacity,mAh,10500.00
2d97bf45-0b0e-419a-a1a3-84065db8c90b,iPad Pro 13-inch 128GB,b584fcf4-6fea-46d7-8a5b-2cc994df30f3,Apple,7,Battery Capacity,mAh,10307.00
50f1b8b6-8ab5-452f-bdc6-7a7c8e790b30,iPad Pro 13-inch 256GB,b584fcf4-6fea-46d7-8a5b-2cc994df30f3,Apple,8,Battery Capacity,mAh,10307.00
1b9349db-709d-46ab-baf4-2c234c4dfa18,iPad Pro 13-inch 512GB,b584fcf4-6fea-46d7-8a5b-2cc994df30f3,Apple,9,Battery Capacity,mAh,10307.00
ccbcbe82-c466-4d8e-b5a3-54a39c49a423,iPad Pro 13-inch 1TB,b584fcf4-6fea-46d7-8a5b-2cc994df30f3,Apple,10,Battery Capacity,mAh,10307.00


In [0]:
%sql select * from gold.segmentation_metrics limit 10

segmentation_code,segmentation_name,range1_name,range2_name,range3_name,range1_qtd,range2_qtd,range3_qtd
46355442-eaf6-4945-b609-87d4681c9aff,Segmentação por RAM (capacidade),Baixa,Média,Alta,185,505,225
bf331dd6-cee7-43fa-9377-cfba9e0f33b6,Segmentação por Preços,< $200,$200-$500,> $500,2551,593,516
ffc7c4e2-dea9-4080-9b07-7fdd3b4f2c06,Segmentação por Tela,Pequena,Média,Grande,29,230,656
